In [1]:
%run EconIndicatorsTraining.ipynb

In [2]:
features_df = getFeatures()
model, results, mse_score = trainEconLinearRegresion(features_df)

In [3]:
recent_data = getAllIndicators(full_country_basket)
recent_data_norm = relativeStrength(recent_data, False)

predictions_df = pd.DataFrame(
    data = model.predict(recent_data_norm), 
    index = recent_data_norm.index, 
    columns=["Predicted Volatility"])

In [47]:
def getWeights(volatility_df, cut_off = 0.75):
    """
    Returns the weighting recommendation for the selected % most stable currencies.
    """
    # Subtracting out the volatility at the selected % mark. All values less than 0 are set to 0
    volatility_df["InvWt"] = volatility_df["Predicted Volatility"].quantile(cut_off) - volatility_df["Predicted Volatility"]
    volatility_df.loc[volatility_df["InvWt"] < 0, "InvWt"] = 0
    
    volatility_df["Basket Weight"] = volatility_df["InvWt"]/volatility_df["InvWt"].sum()
    
    weights_df = volatility_df.drop(columns = ["Predicted Volatility", "InvWt"])
    
    return weights_df